# Import Lib

In [1]:
import open3d as o3d
import numpy as np

import pandas
from sklearn.decomposition import PCA

import math
import copy

In [2]:
# add aixs
aix_points = [[0, 0, 0],
              [0, 0, 100],
              [400, 0, 0],
              [0, 100, 0],
              [-400, 0, 0],]
aix_lines = [[0, 1], # z-aix
             [0, 2], # x-aix
             [0, 3]] # y-aix

colors = [[1,0,1], [0,0,0], [0,0,0]]
aix_line_set = o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(aix_points),
                                    lines=o3d.utility.Vector2iVector(aix_lines))
aix_line_set.colors = o3d.utility.Vector3dVector(colors)

# Read scan model
## import obj file

In [3]:
scan_obj = o3d.io.read_triangle_mesh("./data/femur_half_4.obj")
print(scan_obj)
o3d.visualization.draw_geometries([scan_obj], mesh_show_wireframe=True)

geometry::TriangleMesh with 35027 points and 68786 triangles.


## Scale unit length to 1mm (coordinate 1000x)

In [4]:
points_center = scan_obj.get_center()
print(points_center)
scan_obj.scale(1000.0, points_center)

[ 0.24559977 -0.49631218 -0.23152044]


geometry::TriangleMesh with 35027 points and 68786 triangles.

## change to point cloud

In [5]:
number_of_points = np.asarray(scan_obj.vertices).shape[0]
scan_obj.compute_vertex_normals()
scan_pcd = scan_obj.sample_points_uniformly(number_of_points)

## Delete floor plane

### find plane using RANSAC: plane function: ax + by + cz + d = 0

In [6]:
plane_model, inliers = scan_pcd.segment_plane(distance_threshold=2,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# floor
inlier_cloud = scan_pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# bone 
bone_cloud = scan_pcd.select_by_index(inliers, invert=True)

Plane equation: 0.00x + 1.00y + -0.01z + 6.81 = 0


## Delete outliers

In [7]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [8]:
cl, ind = bone_cloud.remove_statistical_outlier(nb_neighbors=10,
                                                std_ratio=2.0)
# display outliers
display_inlier_outlier(bone_cloud, ind)

Showing outliers (red) and inliers (gray): 


In [9]:
bone_cloud = bone_cloud.select_by_index(ind)

## Calculate PCA

### subtract mean each column

In [10]:
scan_bone_points_cloud_array = np.asarray(bone_cloud.points)
scan_bone_points_cloud_array = scan_bone_points_cloud_array - scan_bone_points_cloud_array.mean(axis=0, keepdims=True)
print(scan_bone_points_cloud_array)

[[ 194.68198022    5.25606899   72.57236095]
 [ 199.18025416    5.2923772    73.2320681 ]
 [ 202.08668649    5.072772     72.93378704]
 ...
 [-155.37024379  -20.44356418  -40.49412191]
 [-153.72207276  -19.91029507  -41.15306923]
 [-156.2378249   -19.59967706  -42.43630974]]


### PCA

In [11]:
pca = PCA(n_components=3)
pca.fit(scan_bone_points_cloud_array)
pca.components_

array([[ 0.98956125, -0.0158688 ,  0.14323657],
       [-0.14116314,  0.0933332 ,  0.98557693],
       [ 0.02900865,  0.99550846, -0.09011883]])

#### right hand rule

In [12]:
x_ = pca.components_[0]
y_ =pca.components_[1]
z_ = np.cross(x_, y_)
tran_matrix = np.array([x_, y_, z_])
tran_matrix

array([[ 0.98956125, -0.0158688 ,  0.14323657],
       [-0.14116314,  0.0933332 ,  0.98557693],
       [-0.02900865, -0.99550846,  0.09011883]])

#### rotate about x-aix

In [13]:
rotate_x_matrix = np.array([[1, 0, 0],
                            [0, math.cos(math.pi), -math.sin(math.pi)],
                            [0, math.sin(math.pi), math.cos(math.pi)]])

In [14]:
bone_after_pca = np.dot(tran_matrix, scan_bone_points_cloud_array.T).T 
bone_after_pca

array([[ 202.96135194,   44.53429158,   -4.33978697],
       [ 207.50658754,   44.55288203,   -4.44696893],
       [ 210.34344047,   43.82812554,   -4.33954252],
       ...,
       [-159.22419671,  -19.88558481,   21.20953977],
       [-157.69607822,  -20.71791737,   20.57147107],
       [-160.37430516,  -21.59852718,   20.21958266]])

In [15]:
bone_after_rotation = np.dot(rotate_x_matrix.T, bone_after_pca.T).T 
bone_after_rotation

array([[ 202.96135194,  -44.53429158,    4.33978697],
       [ 207.50658754,  -44.55288203,    4.44696893],
       [ 210.34344047,  -43.82812554,    4.33954252],
       ...,
       [-159.22419671,   19.88558481,  -21.20953977],
       [-157.69607822,   20.71791737,  -20.57147107],
       [-160.37430516,   21.59852718,  -20.21958266]])

In [16]:
scan_bone_pca_pcd = o3d.geometry.PointCloud()
scan_bone_pca_pcd.points = o3d.utility.Vector3dVector(bone_after_rotation)
o3d.visualization.draw_geometries([scan_bone_pca_pcd, aix_line_set])

## save file

In [17]:
o3d.io.write_point_cloud("./data/measure_whole_femur.ply", scan_bone_pca_pcd)

True